In [1]:
import pandas as pd
from os import walk, listdir

important_filed = ['text', 'content-desc', 'resource-id', 'activity']

In [2]:
def remove_oracles(events):
    return events[events['event_type']!= 'oracle'].reset_index()


def remove_unimportant_columns(events):
    return events[['class', 'content-desc', 'text', 'activity', 'resource-id']]


def add_type(events, type):
    events.rename(columns={'content-desc':'content_desc', 'resource-id': 'id'}, inplace=True)
    for col in events.columns:
        events.rename(columns={col: type + '_' + col}, inplace=True)
    return events


def clean_test(events, type):
    events = remove_oracles(events)
    events = remove_unimportant_columns(events)
    events = add_type(events, type)
    return events

def clean_df(df):
    return df

def get_mig_events(series, type):
    app_name = series[type + '_app']
    events = pd.read_json('atm_gt/' + app_name + '.json').fillna('')
    events = clean_test(events, type)
    events[type + '_app'] = app_name.replace('-','')
    return events



In [12]:
craft_map = pd.read_csv('craft_map.csv')
s_list = []
for i, series in craft_map.iterrows():
    src_events = get_mig_events(series, 'src')
    target_events = get_mig_events(series, 'target')
    src_index= series['src_index']
    target_index= series['target_index']
    src_event = src_events.iloc[src_index]
    target_event = target_events.iloc[target_index]
    row = pd.concat([src_event, target_event], axis=0)
    s_list.append(row)

df = pd.concat(s_list, axis=1).T.reindex()
df = clean_df(df)
df.reset_index(drop=True)

In [9]:
def get_mig_events_atm(series, type):
    app_name = series['src_app'] + '-' +series['target_app']
    events = pd.read_json('atm_gt/' + app_name + '.json').fillna('')
    events = clean_test(events, type)
    events[type + '_app'] = app_name.replace('-','')
    return events

atm_map = pd.read_csv('atm_map.csv')

s_list = []
for i, series in atm_map.iterrows():
    src_events = get_mig_events(series, 'src')
    target_events = get_mig_events_atm(series, 'target')
    src_index= series['src_index']
    target_index= series['target_index']
    try:
        src_event = src_events.iloc[src_index]
    except:
        print('')
    target_event = target_events.iloc[target_index]
    row = pd.concat([src_event, target_event], axis=0)
    s_list.append(row)

df = pd.concat(s_list, axis=1).T.reindex()
df = clean_df(df)
df.reset_index(drop=True)


,src_class,src_content_desc,src_text,src_activity,src_id,src_app,target_class,target_content_desc,target_text,target_activity,target_id,target_app
0,android.widget.ImageButton,,,.view.MainActivity,com.benoitletondor.easybudgetapp:id/fab_expand...,ExpenseTracker1,android.widget.ImageButton,,,.MainActivity,luankevinferreira.expenses:id/fab,ExpenseTracker1ExpenseTracker2
1,android.widget.ImageButton,,,.view.MainActivity,com.benoitletondor.easybudgetapp:id/fab_new_ex...,ExpenseTracker1,android.widget.EditText,,Add a description,.ExpenseActivity,luankevinferreira.expenses:id/expense_description,ExpenseTracker1ExpenseTracker2
2,android.widget.EditText,,,.view.ExpenseEditActivity,com.benoitletondor.easybudgetapp:id/descriptio...,ExpenseTracker1,android.widget.EditText,,0.00,.ExpenseActivity,luankevinferreira.expenses:id/expense_value,ExpenseTracker1ExpenseTracker2
3,android.widget.ImageButton,,,.view.MainActivity,com.benoitletondor.easybudgetapp:id/fab_expand...,ExpenseTracker1,android.widget.ImageButton,,,.MainActivity,com.kvannli .simonkvannli.dailybudget:id/image...,ExpenseTracker1ExpenseTracker3
4,android.widget.ImageButton,,,.view.MainActivity,com.benoitletondor.easybudgetapp:id/fab_new_ex...,ExpenseTracker1,android.widget.EditText,,Note (Optional),.InputActivity,com.kvannli.simonkvannli.dailybudget:id/input_...,ExpenseTracker1ExpenseTracker3
...,...,...,...,...,...,...,...,...,...,...,...,...
112,android.widget.EditText,,"e.g. wishlist, family list",.ShoppingActivity,org.openintents.shopping:id/edittext,ShoppingList4,android.widget.EditText,,,.ui.main.MainActivity,privacyfriendlyshoppinglist.secuso.org.privacy...,ShoppingList4ShoppingList3
113,android.widget.Button,,OK,.ShoppingActivity,android:id/button1,ShoppingList4,android.widget.Button,,OKAY,.ui.main.MainActivity,android:id/button1,ShoppingList4ShoppingList3
114,android.widget.Button,,ADD,.ShoppingActivity,org.openintents.shopping:id/button_add_item,ShoppingList4,android.widget.ImageButton,,,.ui.products.ProductsActivity,privacyfriendlyshoppinglist.secuso.org.privacy...,ShoppingList4ShoppingList3
115,android.widget.Spinner,,,.ShoppingActivity,org.openintents.shopping:id/spinner_listfilter,ShoppingList4,android.widget.EditText,,,.ui.products.ProductsActivity,privacyfriendlyshoppinglist.secuso.org.privacy...,ShoppingList4ShoppingList3
